## import libraries, packages, data

### libraries, packages

In [1]:
#importing libraries & packages
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#display multiple outputs from cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# machine learning
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from imblearn.over_sampling import ADASYN, SMOTE
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import svm
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

Using TensorFlow backend.


In [2]:
# keras
import keras
from keras.models import Sequential 
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle
from sklearn import preprocessing, model_selection
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

### import clean data from part 1

In [38]:
# import Animal Control Incidents df
df = pd.read_csv('a_control.csv')

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37902 entries, 0 to 37901
Data columns (total 11 columns):
species         37902 non-null object
breed           37902 non-null object
size            37902 non-null object
color           37902 non-null object
condition       37902 non-null object
temperment      37902 non-null object
weekday         37902 non-null float64
month           37902 non-null float64
service_type    37902 non-null object
disposition     37902 non-null object
municipality    37902 non-null object
dtypes: float64(2), object(9)
memory usage: 3.2+ MB


In [40]:
df = df[['species', 'breed', 'size', 'color', 'service_type']]

In [41]:
for word in ('trap', 'patrol_area', 'pit', 'special_case'):
    df['service_type'] = df['service_type'].replace(word, 'other')
    
for word in ('injured_animal', 'vet_pickup', 'animal_rescue', 'sick/injured', 'cruelty'):
    df['service_type'] = df['service_type'].replace(word, 'inj/sick/rescue/cruelty')
    
for word in ('dog_fighting', 'dangerous', 'bite_case'):
    df['service_type'] = df['service_type'].replace(word, 'bite/dangerous')
    
for word in ('loose', 'stray', 'trapped_animal', 'wild_life', 'snake'):    
    df['service_type'] = df['service_type'].replace(word, 'loose/stray/trapped/wild')

for word in ('unit_standing_by', 'school_call', 'other', 'owner_signed_release'):
    df['service_type'] = df['service_type'].replace(word, 'owner/sch/unit/other')


for word in ('loose/stray/trapped/wild', 'inj/sick/trapped/wild', 'owner/sch/unit/other','inj/sick/rescue/cruelty'):
    df['service_type'] = df['service_type'].replace(word, 'not_bite/not_dangerous')

In [42]:
# # assign target
y = df["service_type"]

#remove target from features
df.drop(columns= ['service_type'], axis=1, inplace=True)

# dummy categorical features
#col_list = list(df.select_dtypes(include=['object']).columns)
col_list = list(df.columns)

# create dummies for categorical features, assign to X
df = pd.DataFrame(pd.get_dummies(data=df, drop_first=True
                   , prefix=col_list))
X = df

# X_train_d and y_train_d = using original standard test train spit dad
# that way, I can try out different resampling methods
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=33)

### functions

In [43]:
# print model report
def model_report(model, cm = True): 
    
    print('\nCross Validation Scoring:\n')
    
    train_score = model.score(X_train, y_train)
    print("train score: {:.4}%".format(train_score * 100))

    test_score = model.score(X_test, y_test)
    print("test score: {:.4}%".format(test_score * 100))
    
    if cm :

        cv_score = np.mean(cross_val_score(model, X, y, cv=3))
        print("cross val score: {:.4}%".format(cv_score * 100))

        print("\nClassification Report:")
        print(classification_report(actuals, predictions))

        col_list = []
        ind_list = []                     
        for x in y_test.unique():
            col_list.append('pred_'+ x)
            ind_list.append('real_'+x)


        print('\nConfusion matrix:'.format(accuracy_score))
        display(pd.DataFrame(confusion_matrix(y_test, predictions)
                             , columns=col_list, index=ind_list))


    else:
        pass

### Feature importance accoring to chi2 and ExtraTrees

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Feats','Score']  #naming columns
print(featureScores.nlargest(25,'Score'))  #print best features

In [ ]:
## extraTrees classifier feature importance
# # using extratrees to look at feature importance
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(X,y)
#print(model.feature_importances_) 
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(25).plot(kind='barh');
plt.show();

## prep data

### resampling

Due to the imbalance of class data, it could be beneficial to resample to compensate for the missing data. Using smote to oversample, then random undersampling is a common practice in these circumstances



In [44]:
# over = SMOTE(sampling_strategy='minority', random_state=3)
# #under = RandomUnderSampler(sampling_strategy='not minority', random_state=3)
# steps = [('o', over)
# #         , ('u', under)
#         ]
# pipeline = Pipeline(steps=steps)
# # # transform the dataset
# X_train, y_train = pipeline.fit_resample(X_train, y_train)

In [50]:
print (y_train.value_counts())

not_bite/not_dangerous    28864
bite/dangerous            28864
Name: service_type, dtype: int64


## __modeling__

Logistic Regression basic default model

In [47]:
lr = LogisticRegression(random_state=3)
lr.fit(X_train, y_train)
 
# Predict on training set
predictions = lr.predict(X_test)
actuals = y_test

model_report(lr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=3, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


Cross Validation Scoring:

train score: 80.4%
test score: 74.24%
cross val score: 95.2%

Classification Report:
                        precision    recall  f1-score   support

        bite/dangerous       0.11      0.59      0.18       364
not_bite/not_dangerous       0.97      0.75      0.85      7217

              accuracy                           0.74      7581
             macro avg       0.54      0.67      0.51      7581
          weighted avg       0.93      0.74      0.82      7581


Confusion matrix:


,pred_not_bite/not_dangerous,pred_bite/dangerous
real_not_bite/not_dangerous,215,149
real_bite/dangerous,1804,5413


### decision tree

__default model__

In [48]:
# initialize and fit default decision tree
dt_model = DecisionTreeClassifier(random_state=3)
dt_model.fit(X_train, y_train)
predictions = dt_model.predict(X_test)
#actuals = y_test

model_report(dt_model)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=3, splitter='best')


Cross Validation Scoring:

train score: 84.65%
test score: 74.87%
cross val score: 94.62%

Classification Report:
                        precision    recall  f1-score   support

        bite/dangerous       0.11      0.62      0.19       364
not_bite/not_dangerous       0.98      0.76      0.85      7217

              accuracy                           0.75      7581
             macro avg       0.54      0.69      0.52      7581
          weighted avg       0.93      0.75      0.82      7581


Confusion matrix:


,pred_not_bite/not_dangerous,pred_bite/dangerous
real_not_bite/not_dangerous,226,138
real_bite/dangerous,1767,5450


In [ ]:
# set random search params
criterion=['gini','entropy']
max_leaf_nodes = [None]
max_features = ['auto', 'sqrt']
max_depth = [5,10,15,17,20]
min_samples_split = [10,20,30,50,100]
min_samples_leaf = [3,5,7,10,20]
bootstrap = [True, False]

random_grid = {'criterion':criterion,
               'max_leaf_nodes': max_leaf_nodes,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf
               }

# random grid search
dt_random = RandomizedSearchCV(estimator = dt_model
                               , param_distributions = random_grid
                               , n_iter = 100, cv = 3, verbose=3, error_score=0
                               , scoring=('accuracy')
                               , random_state=3, n_jobs = -1)
# fit random search model
dt_random.fit(X_train, y_train);

print('best accuracy: {:.4}%'.format(dt_random.best_score_ * 100));
print(dt_random.best_params_);

__grid search__

In [ ]:
# grid search params
param_grid_dt = {'criterion': ['entropy','gini'],
    'max_depth': [15,17,20],
    'min_samples_split': [2],
    'min_samples_leaf': [6,7,8],
     'max_features':[40,50,60],
     'max_leaf_nodes':[None]
        }

# grid search
gs_dt = GridSearchCV(estimator=dt_model,
                    param_grid=param_grid_dt,
                     error_score=0,
                    scoring=('accuracy'),
                    cv=5, n_jobs=-1, verbose=2)

# fitting grid search
gs_dt.fit(X_train, y_train);

# best parameters
print('Best score: {:.3}%'.format(gs_dt.best_score_ * 100));
print('params:\n', gs_dt.best_params_);

In [ ]:
# using best params to fit model
dt_gs_model = DecisionTreeClassifier(criterion='gini',
                              max_depth=15,
                              max_features=40,
                              max_leaf_nodes= None,
                               min_samples_leaf=7,
                               min_samples_split=2
                                
                                    )
dt_gs_model.fit(X_train, y_train)

predictions = dt_gs_model.predict(X_test)
actuals = y_test

In [ ]:
model_report(dt_gs_model)

### Random Forest

---
__default model__

In [51]:
#initialize and fit random forest
rf_model = RandomForestClassifier(random_state=3)
rf_model.fit(X_train, y_train)

predictions = rf_model.predict(X_test)
#actuals = y_test

model_report(rf_model)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=3, verbose=0,
                       warm_start=False)


Cross Validation Scoring:

train score: 84.65%
test score: 75.04%
cross val score: 94.97%

Classification Report:
                        precision    recall  f1-score   support

        bite/dangerous       0.11      0.62      0.19       364
not_bite/not_dangerous       0.98      0.76      0.85      7217

              accuracy                           0.75      7581
             macro avg       0.54      0.69      0.52      7581
          weighted avg       0.93      0.75      0.82      7581


Confusion matrix:


,pred_not_bite/not_dangerous,pred_bite/dangerous
real_not_bite/not_dangerous,226,138
real_bite/dangerous,1754,5463


___
__Random Grid Search__

The default model is overfitting. Some ways to avoid overfitting with random forests are: increasing the n_estimators, reducing max features, limiting max depth, and increasing min leaf samples can help construct a more robust fit. Being that it did perform well despite the overfitting, I decided not to go crazy playing with the parameters.

In [ ]:
# set random search params

n_estimators = [250,500,750,1000]
# min_weight_fraction_leaf=[0.0, 0.001, 0.0001]
# min_samples_split = [3,4,5,6]
# min_samples_leaf = [2,3]
# min_impurity_split = [0.0]
# min_impurity_decrease = [0.0, 0.0001]
# max_samples = [None, 20,25,30,35,40,50]
# max_leaf_nodes = [None, 3,5,8,10,15,20,25]
# max_features = ['auto', 3,5,6,8]
max_depth = [2,3,5,10,20,30,35,40]
criterion = ['entropy', 'gini']
# bootstrap = [True, False]

# Create the random grid
random_grid = { 'n_estimators': n_estimators,
#                  'min_weight_fraction_leaf': min_weight_fraction_leaf,
#                  'min_samples_split': min_samples_split,
#                  'min_samples_leaf': min_samples_leaf,
#                  'min_impurity_split': min_impurity_split,
#                  'min_impurity_decrease': min_impurity_decrease,
#                  'max_samples': max_samples,
#                  'max_leaf_nodes': max_leaf_nodes,
#                  'max_features': max_features,
                  'max_depth': max_depth,
                 'criterion': criterion,
#                  'bootstrap': bootstrap
              }

# fit and search random param combinations
rf_random = RandomizedSearchCV(estimator = rf_model
                               , param_distributions = random_grid
                               , n_iter = 25
                               , cv = 3
                               , verbose=10
                               , random_state=1
                               , n_jobs = -1);
# Fit the random search model
rf_random.fit(X_train, y_train);

In [ ]:
# print best parameters
print('best accuracy: {:.4}%'.format(rf_random.best_score_ * 100));
print(rf_random.best_params_);

___
__grid search__

In [ ]:
param_grid_rf = {
        'n_estimators': [750,1100]
#          , 'min_samples_leaf': [3,4]
#          , 'min_samples_split':[3,4,5,6]
#         , 'max_samples': [20,25,30]
#         , 'max_leaf_nodes':[7,13,20]
#           , 'max_features': [5,6,7]
         , 'max_depth': [3,5,10,20]
         , 'criterion': ['entropy', 'gini']
#          , 'bootstrap': [False, True]
}

# grid search
gs_rf = GridSearchCV(estimator=rf_model
                    ,param_grid=param_grid_rf
                    ,cv=3
                     ,refit=True
                     ,error_score=0
                     , n_jobs=-1
                     , verbose=10
                    )

# fitting grid search
gs_rf.fit(X_train, y_train);

# best parameters
print('best accuracy: {:.4}%'.format(gs_rf.best_score_ * 100));
print('params:\n', gs_rf.best_params_);

___
__best model__

In [ ]:
# fitting model using best params
rf_gs_model = RandomForestClassifier(
                                   # bootstrap= False
                                    criterion= 'gini'
                                    , max_depth= 20
                                   # , max_features = 'auto'
                                   # , max_leaf_nodes = None
                                   # , max_samples = None
#                                     , min_samples_leaf = 3
                                   # , min_samples_split = 8
                                    , n_estimators = 750
                                    );
rf_gs_model.fit(X_train, y_train);

predictions = rf_gs_model.predict(X_test);

# new model report
model_report(rf_gs_model)

### XGBoost

In [ ]:
# initiate model
xgb_model = XGBClassifier(random_state=3)
xgb_model.fit(X_train, y_train)

predictions = xgb_model.predict(X_test)
# actuals = y_test

In [ ]:
model_report(xgb_model)

In [ ]:
#set random search params
learning_rate=[.15 ,.1,  .05]
#max_depth = [2,3,4,5,6]
min_child_weight = [.5, 1,1.5]
n_estimators = [90,100, 110]
#min_weight_fraction_leaf = [0,0.0001, 0.01, 1]

random_grid = {
               'learning_rate':learning_rate,
               #'max_depth': max_depth,
               'min_child_weight':min_child_weight
              # 'n_estimators': n_estimators
              }

# searching random params
xgb_random = RandomizedSearchCV(estimator = xgb_model
                               , param_distributions = random_grid
                               , n_iter = 5, cv = 3, verbose=10
                               , random_state=3, n_jobs = -1)
# fit random search model
xgb_random.fit(X_train, y_train)
print('best accuracy: {:.4}%'.format(xgb_random.best_score_ * 100));
print(xgb_random.best_params_);

In [ ]:
param_grid_xgb = [
    {
#         'xgb_model__min_samples_split':[2,3],
      #   'xbg_model__gamma':[0.01,0.1,.05],
        'xgb_model__min_child_weight': [1,1.2],
    'xgb_model__learning_rate': [0.009,0.1]
    #'xgb_model__n_estimators': [150,200,250],
    #'xgb_model__max_depth': [2,3]
    }
]

# Construct Grid Search
gs_xgb = GridSearchCV(estimator=xgb_model,
                    param_grid=param_grid_xgb,
                    scoring='accuracy',random_state=3,
                    cv=3, n_jobs=-1, verbose=10)

# Fit using grid search
gs_xgb.fit(X, y)

# Best accuracy and parameters
print('best score: {:.3}%'.format(gs_xgb.best_score_ * 100))
print('params:\n', gs_xgb.best_params_);

In [ ]:
# fit using best params
xgb_gs_model = XGBClassifier(learning_rate=0.009,
                           #  max_depth=3,
                             min_child_weight=1,
                          #   n_estimators=90)
xgb_gs_model.fit(X_train, y_train)

predictions = xgb_gs_model.predict(X_test)

model_report(xgb_gs_model)

### AdaBoost

In [ ]:
#initialize and fit default model
ada_model = AdaBoostClassifier(random_state=3)
ada_model.fit(X_train, y_train)

predictions = ada_model.predict(X_test)
# actuals = y_test

___
__Default Model__

In [ ]:
model_report(ada_model)

__Random Grid Search__

In [ ]:
#set random search params
learning_rate= [.00001,.001,.01,.1,1, 1.5]

n_estimators = [1,5,10,50,100, 200]

random_grid = {'learning_rate':learning_rate,
               'n_estimators': n_estimators
              }

In [ ]:
#random grid search
ada_random = RandomizedSearchCV(estimator = ada_model
                               , param_distributions = random_grid
                               , n_iter = 20, cv = 3, verbose=10
                               , random_state=3, n_jobs = -1)
# Fit the random search model
ada_random.fit(X_train, y_train)

In [ ]:
ada_random.best_params_

___
__grid search__

In [ ]:
#grid search params and fitting grid search
param_grid_ada = [
    {'ada_model__n_estimators': [9,10,11,12,50,60,100],
    'ada_model__learning_rate': [.9,1,1.1,1.5]}
]

# Construct Grid Search
gs_ada = GridSearchCV(estimator=ada_model,
                    param_grid=param_grid_ada,
                    scoring='accuracy',
                    cv=5, n_jobs=-1, verbose=10)

# Fit using grid search
gs_ada.fit(X_train, y_train)

# Best accuracy and parameters
print('best recall: {:.4}%'.format(gs_ada.best_score_ * 100))
print('params:\n', gs_ada.best_params_)

In [ ]:
# fit best params
ada_gs_model = AdaBoostClassifier(n_estimators=100,
                                   learning_rate=1.1)
ada_gs_model.fit(X_train, y_train)

predictions = ada_gs_model.predict(X_test)

__best model__

In [ ]:
model_report(ada_gs_model)

### gradient boost

In [ ]:
#initialize and fit gradient boost
gbt_model = GradientBoostingClassifier(random_state=3)
gbt_model.fit(X_train, y_train)

predictions = gbt_model.predict(X_test)
# actuals = y_test

__default model__

In [ ]:
#baseline model report
model_report(gbt_model)

__random grid search__

In [ ]:
# setting parameters for random search
learning_rate=[.1]
loss = ['deviance', 'exponential']
max_depth = [None,2,3,4] 
n_estimators = [50,100,200]
min_samples_split = [2,3,4]
min_samples_leaf = [1,2,3]
min_weight_fraction_leaf = [0,0.0001, 0.001]

random_grid = {'learning_rate':learning_rate,
               'loss': loss,
               'max_depth': max_depth,
               'n_estimators': n_estimators,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'min_weight_fraction_leaf': min_weight_fraction_leaf
               }

#searching random params
gbt_random = RandomizedSearchCV(estimator = gbt_model
                               , param_distributions = random_grid
                               , n_iter = 20, cv = 3, verbose=10
                               , random_state=3, n_jobs = -1)
# fit random search model
gbt_random.fit(X_train, y_train)

In [ ]:
print('best accuracy: {:.4}%'.format(gbt_random.best_score_ * 100));
print(gbt)_random.best_params_)

__grid search__

In [ ]:
# grid search params and run search
param_grid_gbt = [
    {'gbt_model__n_estimators': [40,60,100,120],
     'gbt_model__min_weight_fraction_leaf':[0.001, 0.01, 0.0001],
    'gbt_model__learning_rate': [0.1, .001],
    'gbt_model__min_samples_split': [2,3,4],
    'gbt_model__min_samples_leaf': [2,3,5,6],
    'gbt_model__max_depth': [2,3]
    }
]

# grid search
gs_gbt = GridSearchCV(estimator=gbt_model,
                    param_grid=param_grid_gbt,
                    scoring='accuracy',
                    cv=3, n_jobs=-1, verbose=10)

# fit grid search
gs_gbt.fit(X_train, y_train)

# best params
print('best accuracy: {:.4}%'.format(gs_gbt.best_score_ * 100))
print('params:\n', gs_gbt.best_params_)

In [ ]:
# fitting best params
gbt_gs_model = GradientBoostingClassifier(learning_rate=0.1,
                                         max_depth=2,
                                         min_samples_leaf=2,
                                         min_samples_split=2,
                                         min_weight_fraction_leaf=0.01,
                                         n_estimators=60)
gbt_gs_model.fit(X_train, y_train)

predictions = gbt_gs_model.predict(X_test)

__best model__

In [ ]:
# new model report
model_report(gbt_gs_model)

### SVM

In [ ]:
svm_model = svm.SVC(random_state=3)
svm_model.fit(X_train, y_train)

predictions = svm_model.predict(X_test)
# actuals = y_test

___
__default model__

In [ ]:
model_report(svm_model, cm=False)

___
__random grid search__

In [ ]:
# set random search params
clf_C = [0.07,0.1,1.0, 1.03]
clf_gamma = [0.005,0.001,0.01, 0.1]
clf_kernel = ['rbf', 'linear']

random_grid = {
               'C': clf_C,
               'gamma': clf_gamma,
               'kernel': clf_kernel
               }

# fit and search random param combinations
svm_random = RandomizedSearchCV(estimator = svm_model
                               , param_distributions = random_grid
                               , n_iter = 12 , cv = 3, verbose=10
                               , random_state=3, n_jobs = -1)
# Fit the random search model
svm_random.fit(X_train, y_train)

In [ ]:
print('best accuracy: {:.4}%'.format(svm_random.best_score_ * 100));
print(svm_random.best_params_)

___
__grid search__

In [ ]:
# Set grid search params
param_grid_svm = [
 
  {'C': [1.0, 1.01, 1.03], 'gamma': [0.008, 0.01, 0.012]
   , 'kernel': ['rbf', 'linear']
  }]
 

# Construct grid search
gs_svm = GridSearchCV(estimator=svm_model,
            param_grid=param_grid_svm,
            cv=5, verbose=10, return_train_score = True)

# Fit using grid search
gs_svm.fit(X_train, y_train)

# Best accuracy
print('best score: {:.4}%'.format(gs_svm.best_score_ * 100));
print('\nBest params:\n', gs_svm.best_params_)

In [ ]:
svm_gs_model = svm.SVC(C=1.03
                       ,gamma=0.008
                        ,kernel='rbf'
                             )
svm_gs_model.fit(X_train, y_train)

predictions = svm_gs_model.predict(X_test)

___
__best model__

In [ ]:
model_report(svm_gs_model, cm=False)